In [1]:
import os
import pandas as pd
from textblob import *
import nltk
import numpy as np
import openpyxl
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import *
'''
    LIAR
'''



data = pd.read_csv(r"../data/Des_fake_news/LIAR_PROCESSED.csv")
data["flag"] = data["class"]
data = data.sample(5000)
data



,Unnamed: 0,id,class,text,title,speaker,job title,state,party,barelytrues,...,text_tb_pol,text_tb_sub,title_tb_pol,title_tb_sub,title_vader_neg,title_vader_neu,title_vader_pos,text_vader_neg,text_vader_neu,text_vader_pos
2718,2718,4594.json,0,Says Rick Perry co chaired Al Gores campaign f...,candidates biography elections,michele-bachmann,Congresswoman,Minnesota,republican,8,...,0.00,0.000000,0.0,0.0,0.000,1.000,0.000,0.000,1.000,0.000
4121,4121,1068.json,1,Every year tobacco kills more Americans than d...,consumer safety,george-will,Columnist,Maryland,columnist,7,...,0.50,0.500000,0.0,0.0,0.000,0.263,0.737,0.492,0.508,0.000
8205,8205,7662.json,0,Says Texas school systems lose days a year on...,education,david-dewhurst,Lieutenant governor,Texas,republican,8,...,0.00,0.000000,0.0,0.0,0.000,1.000,0.000,0.231,0.769,0.000
3822,3822,5981.json,0,On transportation financing,state budget transportation,chris-christie,Governor of New Jersey,New Jersey,republican,10,...,0.00,0.000000,0.0,0.0,0.000,1.000,0.000,0.000,1.000,0.000
6859,6859,5504.json,0,Says Barack Obama promised gasoline at a ga...,energy gas prices transportation,lenny-curry,chairman of the Republican Party of Florida,Florida,republican,2,...,0.00,0.000000,0.0,0.0,0.000,0.588,0.412,0.000,0.737,0.263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4175,4175,608.json,1,We could save all the oil that they re talking...,energy,barack-obama,President,Illinois,democrat,70,...,0.00,0.076923,0.0,0.0,0.000,0.000,1.000,0.000,0.887,0.113
7342,7342,11634.json,1,Twenty five percent of our kids in foster care...,children drugs poverty,vicky-hartzler,U.S. Representative,Missouri,republican,1,...,0.00,0.000000,0.0,0.0,0.623,0.377,0.000,0.000,0.842,0.158
918,918,1978.json,0,On whether a U S Supreme Court nominee should...,kagan nomination supreme court,elena-kagan,U.S. solicitor general,New York,democrat,0,...,0.00,0.000000,0.0,0.0,0.000,0.455,0.545,0.000,0.783,0.217
5762,5762,3441.json,1,Lake Erie supports nearly percent of Ohios jo...,economy environment federal budget tourism,betty-sutton,U.S. representative from Ohio's 13th District,Ohio,democrat,0,...,0.05,0.200000,0.0,0.0,0.000,1.000,0.000,0.000,0.857,0.143


In [ ]:
'''
    Add tf-idf vectorizer
'''

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import pandas as pd

def tf_idf_vectorize(df: pd.DataFrame, corpus: pd.Series, vocabulary: list[str]) -> tuple[list[str], pd.DataFrame]:
    stop_words = stopwords.words("english")
    vectorizer = TfidfVectorizer(strip_accents="ascii", lowercase=True, stop_words=stop_words, max_features=500, ngram_range=(1,3), vocabulary=vocabulary)
    features = vectorizer.fit_transform(corpus).toarray()
    names = vectorizer.get_feature_names_out()
    headers = [f"__word{i}" for i in range(len(names))]
    feature_frame = pd.DataFrame(features, columns=headers)
    #final = pd.concat([df, feature_frame], axis=1)
    return (names, feature_frame)


def get_full_vocabulary(corpus: pd.Series) -> list[str]:
    vectorizer = TfidfVectorizer(strip_accents="ascii", lowercase=True, stop_words=stopwords.words("english"), max_features=500, ngram_range=(1,3))
    vectorizer.fit_transform(corpus).toarray()
    return vectorizer.get_feature_names_out()


def add_tf_idf_vector(x: pd.DataFrame, partial_corpus: pd.Series, full_corpus: pd.Series) -> pd.DataFrame:
    temp = pd.concat((x, partial_corpus), axis=1).reset_index()
    # first, we need a vocabulary from the entire dataset
    vocabulary = get_full_vocabulary(full_corpus)

    tf_idf_vector = tf_idf_vectorize(temp, temp[partial_corpus.name], vocabulary=vocabulary)[1].reset_index()
    temp = temp.reset_index()
    temp = pd.concat((temp, tf_idf_vector), axis=1).drop([partial_corpus.name, "index"], axis=1)
    return temp



In [2]:
from openpyxl import load_workbook, Workbook

'''
    Define filtered dataset, classes, features, dataframe for model accuracies, and excel file for results
'''

'''Drop NA'''
filtered = data.dropna()
classes = filtered["flag"]

filtered["corpus"] = filtered["text"]


# raw polarity and subjectivity scores from Textblob, Vader
features = filtered[["text_tb_pol",	"text_tb_sub","text_vader_neg",	"text_vader_neu",	"text_vader_pos"]]

# dataframe to store accuracies for NN and log regression
#accuracy_df = pd.DataFrame(columns=["polarity",
#               "tfidf",
#               "combined"])


EXCEL_FILE = r"../data/Des_fake_news/Sentiment_Analysis_Results/LIAR_RESULTS.xlsx"
# overwrite book if exists
book = Workbook()
book.save(filename=EXCEL_FILE)
book.close()


NameError: name 'data' is not defined

In [ ]:
'''
    POLARITY + TF-IDF
'''
from sklearn.discriminant_analysis import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import KFold
import openpyxl.drawing
from sklearn import metrics
import matplotlib.pyplot as plt
import io



# average accuracies
comb_output = {"train" : None, "test" : None}
comb_matrices = []


comb_fpr = []
comb_tpr = []
comb_thresh = []

comb_auc = 0


train_total = 0
test_total = 0
run_count = 0


#lbgfs
x = features
y = classes
folds = 5
kf = KFold(n_splits=folds)

for i, (train_index, test_index) in enumerate(kf.split(x)):
    print(i)
    scaler = StandardScaler()
    print("tf_idf training")
    x_train = add_tf_idf_vector(x.iloc[train_index], filtered["corpus"].iloc[train_index], filtered["corpus"])
    x_train = scaler.fit_transform(x_train)

    print("tf_idf testing")
    x_test = add_tf_idf_vector(x.iloc[test_index], filtered["corpus"].iloc[test_index], filtered["corpus"])
    x_test = scaler.transform(x_test)
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]

    comb_ensemble = VotingClassifier(estimators=[
    ('lr', LogisticRegression(max_iter=500)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svc', SVC(kernel='rbf', probability=True, random_state=42))
    ], voting='soft')
    print("ensemble fitting")
    comb_ensemble.fit(x_train, y_train)
    y_pred = comb_ensemble.predict(x_test)

    fpr, tpr, thresh = metrics.roc_curve(y_test, y_pred, pos_label=1)
    comb_fpr.append(fpr)
    comb_tpr.append(tpr)
    comb_thresh.append(thresh)
    comb_auc += metrics.auc(fpr, tpr)

    comb_matrices.append(confusion_matrix(y_test, y_pred))
    test_total += accuracy_score(y_test, y_pred)

    y_train_pred = comb_ensemble.predict(x_train)
    train_total += accuracy_score(y_train, y_train_pred)

    run_count += 1

print(f"Train accuracy on {folds}-fold cross-validation: {train_total / float(run_count)}")
print(f"Test accuracy on {folds}-fold cross-validation: {test_total / float(run_count)}")

comb_output["test"] = [(test_total / run_count)]
comb_output["train"] = [(train_total / run_count)]
comb_output = pd.DataFrame(comb_output)

print("Average confusion matrix:")
comb_matrices = np.mean(np.array(comb_matrices), axis=0)
print(comb_matrices)


comb_fpr = np.mean(np.array(comb_fpr), axis=0)
comb_tpr = np.mean(np.array(comb_tpr), axis=0)
comb_thresh = np.mean(np.array(comb_thresh), axis=0)

comb_auc = comb_auc / float(run_count)










comb_output

0
tf_idf training
tf_idf testing
ensemble fitting
1
tf_idf training
tf_idf testing
ensemble fitting
2
tf_idf training
tf_idf testing
ensemble fitting
3
tf_idf training
tf_idf testing
ensemble fitting
4
tf_idf training
tf_idf testing
ensemble fitting


In [ ]:
'''
    POLARITY
'''
from sklearn.discriminant_analysis import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import KFold


# average accuracies
pol_output = {"train" : None, "test" : None}
pol_matrices = []


pol_fpr = []
pol_tpr = []
pol_thresh = []

pol_auc = 0

train_total = 0
test_total = 0
run_count = 0


#lbgfs
x = features
y = classes
folds = 5
kf = KFold(n_splits=folds)

for i, (train_index, test_index) in enumerate(kf.split(x)):
    print(i)
    scaler = StandardScaler()
  
    x_train = x.iloc[train_index]
    x_train = scaler.fit_transform(x_train)

    x_test = x.iloc[test_index]
    x_test = scaler.transform(x_test)
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]

    pol_ensemble = VotingClassifier(estimators=[
    ('lr', LogisticRegression(max_iter=500)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svc', SVC(kernel='rbf', probability=True, random_state=42))
    ], voting='soft')
    print("ensemble fitting")
    pol_ensemble.fit(x_train, y_train)
    y_pred = pol_ensemble.predict(x_test)
    fpr, tpr, thresh = metrics.roc_curve(y_test, y_pred, pos_label=1)
    pol_fpr.append(fpr)
    pol_tpr.append(tpr)
    pol_thresh.append(thresh)
    pol_auc += metrics.auc(fpr, tpr)

    pol_matrices.append(confusion_matrix(y_test, y_pred))
    test_total += accuracy_score(y_test, y_pred)

    y_train_pred = pol_ensemble.predict(x_train)
    train_total += accuracy_score(y_train, y_train_pred)

    run_count += 1

print(f"Train accuracy on {folds}-fold cross-validation: {train_total / float(run_count)}")
print(f"Test accuracy on {folds}-fold cross-validation: {test_total / float(run_count)}")

pol_output["test"] = [(test_total / run_count)]
pol_output["train"] = [(train_total / run_count)]
pol_output = pd.DataFrame(pol_output)

print("Average confusion matrix:")
pol_matrices = np.mean(np.array(pol_matrices), axis=0)
print(pol_matrices)

pol_fpr = np.mean(np.array(pol_fpr), axis=0)
pol_tpr = np.mean(np.array(pol_tpr), axis=0)
pol_thresh = np.mean(np.array(pol_thresh), axis=0)

pol_auc = pol_auc / float(run_count)




pol_output

In [ ]:
'''
    TF-IDF
'''
from sklearn.discriminant_analysis import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import KFold


# average accuracies
tf_output = {"train" : None, "test" : None}
tf_matrices = []

train_total = 0
test_total = 0
run_count = 0

tf_fpr = []
tf_tpr = []
tf_thresh = []

tf_auc = 0

#lbgfs
x = features
y = classes
folds = 5
kf = KFold(n_splits=folds)

for i, (train_index, test_index) in enumerate(kf.split(x)):
    print(i)
    scaler = StandardScaler()
    print("tf_idf training")
    x_train = add_tf_idf_vector(x.iloc[train_index], filtered["corpus"].iloc[train_index], filtered["corpus"]).drop(features, axis=1)
    x_train = scaler.fit_transform(x_train)

    print("tf_idf testing")
    x_test = add_tf_idf_vector(x.iloc[test_index], filtered["corpus"].iloc[test_index], filtered["corpus"]).drop(features, axis=1)
    x_test = scaler.transform(x_test)
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]

    tf_ensemble = VotingClassifier(estimators=[
    ('lr', LogisticRegression(max_iter=500)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svc', SVC(kernel='rbf', probability=True, random_state=42))
    ], voting='soft')
    print("ensemble fitting")
    tf_ensemble.fit(x_train, y_train)
    y_pred = tf_ensemble.predict(x_test)
    fpr, tpr, thresh = metrics.roc_curve(y_test, y_pred, pos_label=1)
    tf_fpr.append(fpr)
    tf_tpr.append(tpr)
    tf_thresh.append(thresh)
    tf_auc += metrics.auc(fpr, tpr)


    tf_matrices.append(confusion_matrix(y_test, y_pred))
    test_total += accuracy_score(y_test, y_pred)

    y_train_pred = tf_ensemble.predict(x_train)
    train_total += accuracy_score(y_train, y_train_pred)

    run_count += 1

print(f"Train accuracy on {folds}-fold cross-validation: {train_total / float(run_count)}")
print(f"Test accuracy on {folds}-fold cross-validation: {test_total / float(run_count)}")

tf_output["test"] = [(test_total / run_count)]
tf_output["train"] = [(train_total / run_count)]
tf_output = pd.DataFrame(tf_output)

print("Average confusion matrix:")
tf_matrices = np.mean(np.array(tf_matrices), axis=0)
print(tf_matrices)

tf_fpr = np.mean(np.array(tf_fpr), axis=0)
tf_tpr = np.mean(np.array(tf_tpr), axis=0)
tf_thresh = np.mean(np.array(tf_thresh), axis=0)

tf_auc = tf_auc / float(run_count)



tf_output

In [ ]:
'''
    combine accuracies into one table
'''

final_results = pd.concat((pol_output, tf_output, comb_output), axis=0)
final_results.index = ["Polarity", "Tf-Idf", "Polarity + Tf-Idf"]
final_results

In [ ]:
'''
    Finally, save accuracy metrics to the spreadsheet
'''




book = load_workbook(EXCEL_FILE)
writer = pd.ExcelWriter(EXCEL_FILE, engine="openpyxl")
writer.book = book
final_results.to_excel(writer, sheet_name=f"accuracies")
book.save(filename=EXCEL_FILE)
book.close()



In [ ]:
import openpyxl.drawing
from sklearn import metrics
import matplotlib.pyplot as plt
import io


plt.figure()
lw = 2
plt.plot(pol_fpr, pol_tpr,
 lw=lw, label='Polarity: (%0.2f)' % pol_auc)
plt.plot(tf_fpr, tf_tpr,
 lw=lw, label='Tf-Idf: (%0.2f)' % tf_auc)
plt.plot(comb_fpr, comb_tpr,
 lw=lw, label='Polarity + Tf-Idf: (%0.2f)' % comb_auc)
plt.plot([0, 1], [0, 1], color='black', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic (ROC)')
plt.legend(loc="lower right")
#plt.show()


# save figure as PNG
png = io.BytesIO()
plt.savefig(png, format="png")


# write PNG to excel file
book = load_workbook(EXCEL_FILE)
ws = book.active

img = openpyxl.drawing.image.Image(png)
img.anchor = "A1"
ws.add_image(img)
book.save(filename=EXCEL_FILE)
plt.close()
book.close()

